# Машинное обучение и майнинг данных
## 26/01/2017 Анализ данных в Pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16,8)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

# Pandas
## Основные структуры

Модуль `pandas` существенно упрощает исследование табличных данных в `python`. Работа в нем во многом напоминает работу с таблицами в SQL с тем отличием, что в `pandas` ~~тебе не хочется рвать волосы на голове~~ это делать гораздо удобнее, и в нем заложены некоторые дополнительные инструменты по работе с данными.

### Series

Основными структурами являются `Series` и `DataFrame`. <br/>
`Series` – это проиндексированный одномерный массив значений. Он представляет из себя некий компромис между массивами `numpy` и простым словарем `dict`.

Задать `Series` можно многими способами, например с помощью массива:

In [ ]:
ser = pd.Series(np.random.rand(5))

In [ ]:
ser

Колонка слева - это (строчный) индекс - некоторая нумерация записанных значений

In [ ]:
ser.index

In [ ]:
ser.values

In [ ]:
# Достучаться до одного значения можно так
ser[1]

In [ ]:
# Можно так - это обычная интервальная индексация в python.
ser[0:2]

Но про то, как улучше находить нужные вам значения - чуть позже.

Индексом может быть что угодно, например:

In [ ]:
ser = pd.Series(np.random.rand(5), index=['m', 'i', 'n', 'o', 'r'])

In [ ]:
ser

In [ ]:
ser['r']

In [ ]:
ser['n':'o']

### Индексация

Индексация в `pandas` может временами может показаться запутанной

In [ ]:
ser = pd.Series(np.random.rand(5), index=[1,3,5,6,9])
ser

In [ ]:
ser[3]

In [ ]:
ser[3:5] #?!

Поэтому придумали разные операторы для индексирования, чтобы можно было явно указать, когда вы хотите использовать значения индекса, а когда позицию.

In [ ]:
ser.loc[3:5, ]

In [ ]:
ser.loc[:, ]

In [ ]:
idx = ser > 0.5
ser.loc[idx, ]

In [ ]:
ser.iloc[3:5, ]

Их же используйте для присваивания!!

In [ ]:
idx = ser > 0.5
ser.loc[idx, ] = 0.5
ser

### DataFrame

`DataFrame` — это проиндексированный многомерный массив значений, соответственно каждый столбец `DataFrame`, является структурой `Series`. Индексирование в `DataFrame` ровно тоже, что и в `Series`, с тем отличием, что добавляется второе измерение.

In [ ]:
df = pd.DataFrame(np.random.randn(10, 3),
                  index=range(10),
                  columns=['A', 'B', 'C'])

In [ ]:
df.head() # выводит первые 5 (по-умолчанию) строк таблицы

In [ ]:
print df.index
print df.columns

In [ ]:
df.loc[1:3, ['A', 'B']]

In [ ]:
df.iloc[1:3, 0:2]

`DataFrame` тоже можно транспонировать!

In [ ]:
df.T

In [ ]:
# Краткая описательная статистика
df.describe() 

# Кстати, это тоже DataFrame

Аггрегация в DataFrame (по-умолчанию) происходит по стоблцам

In [ ]:
df.mean()

In [ ]:
df.A.mean()

Перевод данных в нужный тип

In [ ]:
df.A = df.A.astype(int)
df.head()

In [ ]:
print 'Количество уникальных значений в столбце А = %d' % df.A.nunique()

In [ ]:
print 'Самые большие значения в стоблце B :'
print df.B.nlargest(2)

# Гораздо быстрее, чем df.B.sort(ascending=False).iloc[:2]

Важно следить за данными, которые у вас хранятся в `DataFrame`

In [ ]:
df.dtypes

In [ ]:
df.loc[0, 'A'] = 'lalaley'
df.head()

In [ ]:
df.A

Удаление\добавление строк\столбцов

In [ ]:
df.head()

In [ ]:
df.drop(0, axis=0)
# Пока df не изменился !

In [ ]:
df.drop('A', axis=1)

In [ ]:
df.loc[:, 'D'] = np.nan
df.head()

In [ ]:
df.loc[10, :] = 0
df.tail()

## Продолжим обучение Pandas на реальных данных

### Удотребление ЛСД и успеваемость

В 1968 году была [опубликована](http://www.ncbi.nlm.nih.gov/pubmed/5676802) статья под интригующем названием Correlation of Performance Test Scores with Tissue Concentration of Lysergic Acid Diethylamide in Human Subjects.

К статье приложен небольшой набор [данных](https://www.dropbox.com/s/ui14yeeckbc6z7c/drugs-and-math.csv?dl=0), состоящий из 7 наблюдений

In [ ]:
df = pd.read_csv('drugs-and-math.csv', 
                 index_col=0, 
                 sep=',')

In [ ]:
df.head()

In [ ]:
print df.shape
print df.columns
print df.index

Таблица уже отсортирована по колонке Drugs - отсортируем по колонке Score

In [ ]:
df = df.sort_values('Score', 
                    ascending=False)

In [ ]:
df.head()

In [ ]:
df.describe().T # Иногда так лучше

In [ ]:
df.plot(kind='box')

In [ ]:
df.Drugs.hist()

In [ ]:
# df.plot(x='Drugs', y='Score')
df.plot(x='Drugs', y='Score', kind='scatter')

Мы явно видим тенденцию..

### Рождаемость в США

Загрузите два набора данных с информацией о рождаемости в США: [Набор 1](https://www.dropbox.com/s/4v743y3e25lz0an/US_births_1994-2003_CDC_NCHS.csv?dl=0), [Набор 2](https://www.dropbox.com/s/3aoulbiuomamay6/US_births_2000-2014_SSA.csv?dl=0)

In [ ]:
df1 = pd.read_csv('US_births_1994-2003_CDC_NCHS.csv')
df2 = pd.read_csv('US_births_2000-2014_SSA.csv')

`Pandas` обладает мощным функционалом для работы с временными рядами. Самое главное, пожалуй правильно задать столбец с датой в DataFrame. Вернемся к данным:

In [ ]:
df1.head()

Функция `pd.to_datetime()` преобразовывать сырые данные с специальный тип Timestamp. Делать это можно одним из нескольких способов:
* Задан столбец, в котором дата записана в виде строчки произвольного формата (например 2016-03-01, 01032016, 01.03.16 ...)
    
    В этом случае вам надо указать в `pd.to_datetime()` [формат](http://strftime.org/) даты


In [ ]:
print pd.to_datetime('2016-03-01', format='%Y-%m-%d')
print pd.to_datetime('01032016', format='%d%m%Y')
print pd.to_datetime('01-mar-2016', format='%d-%b-%Y')

* Задано несколько столбцов, скаждый из которых содержит элемент даты (столбец под день, месяц, год)

    Тогда, `pd.to_datetime()` подается 3 столбца из таблицы, которые должны **обязательно** именоваться `['year', 'month', 'day']`

In [ ]:
# переименовываем столбец
df1 = df1.rename(columns={'date_of_month': 'day'})

df1.loc[:, 'date'] = \
pd.to_datetime(df1.loc[:, ['year', 'month', 'day']])
df1.head()

Довольно часто информация о дате передается в формате [unix-timestamp](http://www.onlineconversion.com/unix_time.htm).

В этом случае можно воспользоваться функцией `pd.Timestamp.fromtimestamp()`

In [ ]:
pd.Timestamp.fromtimestamp(1453766400)

Давайте сделаем дату индексом в нашей таблице.

In [ ]:
df1 = df1.set_index('date')
df1.head()

Это дает нам возможность выбирать строчки по нужным датам...

In [ ]:
df1.loc['1994-01-01', :]

In [ ]:
df1.loc['1994-01-01':'1995-01-01', :]

.. и не только, например

In [ ]:
# Аггрегация с нужной частотой
df_aggr = df1.resample(rule='AS')\
             .sum()

In [ ]:
df_aggr.head()

In [ ]:
# Визуализация
df1.births.plot(label=u'Исходный ряд')

**Задание**<br/>
Преобразуйте дату в таблице df2 аналогично df1. Становится ясно, что периоды данных в этих файлах пересекаются.

Необходимо проверить, что показатели рождаемости слабо отличаются между двумя файлами. Как бы вы это сделали?

In [ ]:
## Your code here

**Задание**<br/>
Теперь надо аккуратно объединить таблицы, так чтобы соблюсти целостность информации.

In [ ]:
## Your code here

**Задание**<br/>
Найдите количество детей, рождающихся 6, 13 и 20 числа каждого месяца с учетом дня недели.

Выделяется ли как-то пятница 13?

In [ ]:
## Your code here

### Качество вина

Загрузите [датасет](https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv) с информацией о характеристиках вина и его качестве.

In [ ]:
## Your code here

**Задание**<br/>
* Что из себя представляет объект в этом наборе данных? Сколько их?
* Какие признаки описывают объекты? Сколько их?
* Какой признак является целевым?
* Каковы их области значений?
* Есть ли пропуски?

In [ ]:
## Your code here

**Задание**<br/>
Какие признаки больше всего влияют на целевую переменную?

In [ ]:
## Your code here

**Задание**<br/>
Создайте новый столбец `quality_cat`, которая будет иметь значение `"good"` если `quality > 5` и `"bad"` - иначе. <br/>

In [ ]:
## Your code here

**Задание**<br/>
Нарисуйте гистрограммы признака alcohol в группах с `quality_cat == "good"` и `quality_cat == "bad"`.

In [ ]:
## Your code here

**Задание**<br/>
Можете ли вы придумать правило для классификации вина на хорошее и плохое по рисунку выше? Пусть это будет нашей первой моделью)

Напишите функцию `brute_clf_train()` которая бы перебирала пороговое значение по признаку `alcohol` и находило бы "оптимальное" (кстати, что значит оптимальное?)

In [ ]:
## Your code here

**Задание**<br/>
Напишите функцию `brute_clf_predict()` которая бы по значению признака `alcohol` и найденному выше порогу говорила какое качество у вина.

А заодно выводила бы количество "ошибок" на текущем наборе данных

In [ ]:
## Your code here

Проверим, как обобщается наша модель на другие данные.

* Загрузите другой [датасет](https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv)
* Выполните те же панипуляции с признаками
* Используйте нашу простейшую модель для предсказания качества на новых данных

In [ ]:
## Your code here

# Заключение
Естественно, за сегодня мы не разобрали все. Мы очень ограничены во времени и естественно, чтобы стать гуру pandas и numpy нужно постоянно практиковаться, использовать инструмент в повседневных задачах.

**Полезные ссылки**
* [Видеолекции про pandas](http://www.dataschool.io/easier-data-analysis-with-pandas/)
* [Свод туториалов по эффективному использованию pandas](https://github.com/TomAugspurger/modern-pandas)
* [Cheat-sheet](https://www.dropbox.com/s/4eauvhj5tyk3r83/Quandl%2B_%2BPandas%2C%2BSciPy%2C%2BNumPy%2BCheat%2BSheet.pdf?dl=0)

В следующий раз мы начнем говорить по сути - о подготовке признакового описания объектов и о методах машинного обучения. А пока, на затравочку, можно изучить вот такое вот [чтиво](http://martin.zinkevich.org/rules_of_ml/rules_of_ml.pdf)